In [1]:
seed = 3906303

In [2]:
import numpy as np
import pandas as pd
import gzip
import random
np.random.seed(seed)
random.seed(seed)

## The cell below creates a file with all blood glucose measurements from the ICU

In [10]:
icu_file_path = 'mimic-iv-3.1/mimic-iv-3.1/icu/chartevents.csv.gz'
output_path = "bg_filtered_chartevents.csv"

with gzip.open(icu_file_path, mode="rt", encoding="utf-8") as infile, \
     open(output_path, "w", encoding="utf-8") as outfile:

    header = next(infile)  # read and write header
    outfile.write(header)

    for line in infile:
        # Checks if itemid corresponds to blood glucose measurements denoted by 4 values. These 4 values are the 
        # blood glucose measurements we will use to predict dysglycemia
        if line.split(',')[6] in ['220621', '225664', '226537', '228388']:
            outfile.write(line)

In [12]:
df_icu_filtered = pd.read_csv(output_path)
df_icu_filtered

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,10000032,29079034,39553978,NaN,2180-07-23 21:45:00,2180-07-23 22:30:00,220621,115.0,115.0,mg/dL,1
1,10000690,25860671,37081114,84407.0,2150-11-04 10:00:00,2150-11-04 09:39:00,225664,117.0,117.0,NaN,0
2,10000690,25860671,37081114,NaN,2150-11-03 02:56:00,2150-11-03 04:22:00,220621,77.0,77.0,mg/dL,0
3,10000690,25860671,37081114,NaN,2150-11-04 03:03:00,2150-11-04 03:44:00,220621,84.0,84.0,mg/dL,0
4,10000690,25860671,37081114,NaN,2150-11-04 17:54:00,2150-11-04 18:41:00,220621,120.0,120.0,mg/dL,1
...,...,...,...,...,...,...,...,...,...,...,...
1814457,19999987,23865745,36195440,68756.0,2145-11-04 02:00:00,2145-11-04 05:07:00,225664,112.0,112.0,NaN,0
1814458,19999987,23865745,36195440,69532.0,2145-11-03 08:00:00,2145-11-03 07:58:00,225664,122.0,122.0,NaN,0
1814459,19999987,23865745,36195440,96092.0,2145-11-04 08:00:00,2145-11-04 08:11:00,225664,127.0,127.0,NaN,0
1814460,19999987,23865745,36195440,NaN,2145-11-03 01:35:00,2145-11-03 02:42:00,220621,113.0,113.0,mg/dL,1


In [14]:
bg_icu = df_icu_filtered.copy()

In [20]:
bg_icu['charttime'] = pd.to_datetime(
    bg_icu['charttime'],
    format='%Y-%m-%d %H:%M:%S',   # matches 2180-07-23 21:45:00
    errors='raise'               # or 'coerce' to turn bad rows into NaT
)
bg_icu['storetime'] = pd.to_datetime(
    bg_icu['storetime'],
    format='%Y-%m-%d %H:%M:%S',   # matches 2180-07-23 21:45:00
    errors='raise'               # or 'coerce' to turn bad rows into NaT
)

In [17]:
bg_icu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1814462 entries, 0 to 1814461
Data columns (total 11 columns):
 #   Column        Dtype         
---  ------        -----         
 0   subject_id    int64         
 1   hadm_id       int64         
 2   stay_id       int64         
 3   caregiver_id  float64       
 4   charttime     datetime64[ns]
 5   storetime     object        
 6   itemid        int64         
 7   value         float64       
 8   valuenum      float64       
 9   valueuom      object        
 10  warning       int64         
dtypes: datetime64[ns](1), float64(3), int64(5), object(2)
memory usage: 152.3+ MB


In [13]:
df_icu_filtered.describe()

,subject_id,hadm_id,stay_id,caregiver_id,itemid,value,valuenum,warning
count,1.814462e+06,1.814462e+06,1.814462e+06,1.048430e+06,1.814462e+06,1.814462e+06,1.814462e+06,1.814462e+06
mean,1.500162e+07,2.499619e+07,3.497557e+07,4.781917e+04,2.241370e+05,2.091426e+02,2.091426e+02,3.592740e-01
std,2.893341e+06,2.868619e+06,2.891540e+06,2.852588e+04,2.439564e+03,7.727934e+03,7.727934e+03,4.797878e-01
min,1.000003e+07,2.000009e+07,3.000015e+07,4.500000e+01,2.206210e+05,-1.240000e+02,-1.240000e+02,0.000000e+00
25%,1.249285e+07,2.250205e+07,3.247011e+07,2.132000e+04,2.206210e+05,1.100000e+02,1.100000e+02,0.000000e+00
50%,1.501929e+07,2.502270e+07,3.496694e+07,4.635400e+04,2.256640e+05,1.350000e+02,1.350000e+02,0.000000e+00
75%,1.751632e+07,2.744633e+07,3.747037e+07,7.244700e+04,2.256640e+05,1.720000e+02,1.720000e+02,1.000000e+00
max,1.999999e+07,2.999983e+07,3.999986e+07,9.992300e+04,2.265370e+05,1.653550e+06,1.653550e+06,1.000000e+00


### Determining most frequently recorded measurements in ICU

In [5]:
df_icu_labdict = pd.read_csv('mimic-iv-3.1/mimic-iv-3.1/icu/d_items.csv.gz', compression='gzip')
df_icu_labdict.shape

(4095, 9)

In [6]:
df_icu_labdict.sample(5)

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
107,220468,Sorbitol,Sorbitol,ingredientevents,Ingredients - general (Not In Use),mg,Ingredient,NaN,NaN
1754,226108,CCO PAC placed in outside facility,CCO PAC placed in outside facility,chartevents,Access Lines - Invasive,NaN,Checkbox,NaN,NaN
2743,228204,Tandem Heart Access Line Catheter Length,Tandem Heart Access Line Catheter Length,chartevents,Access Lines - Invasive,cm,Numeric,NaN,NaN
2702,228162,Impella Catheter Position,Impella Catheter Position,chartevents,Access Lines - Invasive,cm,Numeric,NaN,NaN
3363,229051,Incision Drainage Amount #9,Incision Drainage Amount #9,chartevents,Skin - Incisions,NaN,Text,NaN,NaN


In [8]:
import gzip, csv
from collections import Counter
import pandas as pd

icu_file_path = 'mimic-iv-3.1/mimic-iv-3.1/icu/chartevents.csv.gz'   # adjust to your tree
item_counts   = Counter()                               # {itemid: n_rows}

# stream through the file and update counts
with gzip.open(icu_file_path, mode="rt", encoding="utf-8") as f:
    reader = csv.DictReader(f)          # handles header automatically
    for row in reader:                  # one row at a time, < 1 kB each
        itemid = int(row["itemid"])     # cast once to int for speed
        item_counts[itemid] += 1

# put counts into a small dataframe
counts_df = (
    pd.DataFrame
      .from_dict(item_counts, orient="index", columns=["n_chartevents"])
      .reset_index()
      .rename(columns={"index": "itemid"})
)

# Left join with dictionary
merged = (
    df_icu_labdict                                       # your reference dict
      .merge(counts_df, on="itemid", how="left")         # keep all ICU items
      .fillna({"n_chartevents": 0})                      # item never found -> 0
      .astype({"n_chartevents": "int"})
)
merged.to_csv("icu_labdict_with_counts.csv", index=False)
merged

NameError: name 'df_icu_labdict' is not defined

In [10]:
# duplicated snippet to show output without error:
merged = (
    df_icu_labdict                                       # your reference dict
      .merge(counts_df, on="itemid", how="left")         # keep all ICU items
      .fillna({"n_chartevents": 0})                      # item never found -> 0
      .astype({"n_chartevents": "int"})
)
merged.to_csv("icu_labdict_with_counts.csv", index=False)
merged

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue,n_chartevents
0,220001,Problem List,Problem List,chartevents,General,NaN,Text,NaN,NaN,583305
1,220003,ICU Admission date,ICU Admission date,datetimeevents,ADT,NaN,Date and time,NaN,NaN,0
2,220045,Heart Rate,HR,chartevents,Routine Vital Signs,bpm,Numeric,NaN,NaN,8752069
3,220046,Heart rate Alarm - High,HR Alarm - High,chartevents,Alarms,bpm,Numeric,NaN,NaN,843381
4,220047,Heart Rate Alarm - Low,HR Alarm - Low,chartevents,Alarms,bpm,Numeric,NaN,NaN,843694
...,...,...,...,...,...,...,...,...,...,...
4090,230172,Patient Reversed,Patient Reversed,procedureevents,3-Significant Events,None,Processes,NaN,NaN,0
4091,230173,Patient - Fast Track Protocol,Patient - Fast Track Protocol,procedureevents,3-Significant Events,None,Processes,NaN,NaN,0
4092,230174,Nerve block in OR,Nerve block in OR,procedureevents,3-Significant Events,None,Processes,NaN,NaN,0
4093,230176,IUC Stabilization Device,IUC Stabilization Device,chartevents,GI/GU,NaN,Checkbox,NaN,NaN,0


In [8]:
merged = pd.read_csv('icu_labdict_with_counts.csv')

Display top 20 most frequently recorded (numeric) measurements in chartevents dataset.

In [9]:
merged[merged['param_type']=='Numeric'].sort_values(by=['n_chartevents'], ascending=False).head(20)

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue,n_chartevents
2,220045,Heart Rate,HR,chartevents,Routine Vital Signs,bpm,Numeric,NaN,NaN,8752069
28,220210,Respiratory Rate,RR,chartevents,Respiratory,insp/min,Numeric,NaN,NaN,8636655
36,220277,O2 saturation pulseoxymetry,SpO2,chartevents,Respiratory,%,Numeric,NaN,NaN,8567015
24,220179,Non Invasive Blood Pressure systolic,NBPs,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN,5378740
25,220180,Non Invasive Blood Pressure diastolic,NBPd,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN,5377689
26,220181,Non Invasive Blood Pressure mean,NBPm,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN,5372922
8,220052,Arterial Blood Pressure mean,ABPm,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN,3096934
6,220050,Arterial Blood Pressure systolic,ABPs,chartevents,Routine Vital Signs,mmHg,Numeric,90.0,140.0,3087686
7,220051,Arterial Blood Pressure diastolic,ABPd,chartevents,Routine Vital Signs,mmHg,Numeric,60.0,90.0,3087261
337,223761,Temperature Fahrenheit,Temperature F,chartevents,Routine Vital Signs,°F,Numeric,NaN,NaN,2055040


# Datagen section

Below we manually select a list of measurements to use for deep learning training. We select just 9 to limit the amount of training data so it can be loaded into memory on Google Colab or our local machines.

In [12]:
training_itemids = {"hr": 220045, # Heartrate
                    "rr": 220210, # Respiratory Rate
                    'spo2': 220277, # O2 saturation pulseoxymetry
                    'nbpm': 220181, # Non Invasive Blood Pressure mean
                    'abpm': 220052, # Arterial Blood Pressure mean
                    'f': 223761, # Temperature Fahrenheit
                    'cvp': 220074, # Central Venous Pressure
                    'hrl': 220047, # Heart Rate Alarm - Low
                    'hrh': 220046, # Heart rate Alarm - High
                   }

In [14]:
list(training_itemids.keys())

['hr', 'rr', 'spo2', 'nbpm', 'abpm', 'f', 'cvp', 'hrl', 'hrh']

The below code goes through chartevents, creates a file for each measurement, and writes data to each particular file if it is a measurement included in 'training_itemids'.

In [17]:
icu_file_path = 'mimic-iv-3.1/mimic-iv-3.1/icu/chartevents.csv.gz'
out_fhs = {
    name: open(f"{name}_filtered_chartevents.csv", "w", encoding="utf-8")
    for name in training_itemids
}

with gzip.open(icu_file_path, mode="rt", encoding="utf-8") as infile:
    header = next(infile)                      # read header once
    for fh in out_fhs.values():                # write header to every file
        fh.write(header)

    for line in infile:
        item_id = line.split(',')[6]           # 7th column = itemid

        # check if this itemid is one we're keeping
        for name, iid in training_itemids.items():
            if item_id == str(iid):
                out_fhs[name].write(line)
                break                          # done with this row

# close files
for fh in out_fhs.values():
    fh.close()

In [3]:
import glob, os

filtered_tables = {
    os.path.basename(fp).split('_')[0]: pd.read_csv(fp)
    for fp in glob.glob('Datagen/*_filtered_chartevents.csv')
}

In [4]:
filtered_tables['abpm']['stay_id'].unique().dtype

dtype('int64')

In [5]:
org_size = []
for name, df in filtered_tables.items():
    print(name, df.shape)
    org_size.append(df.shape[0])

abpm (3096934, 11)
bg (1814462, 11)
cvp (993375, 11)
f (2055040, 11)
hrh (843381, 11)
hrl (843694, 11)
hr (8752069, 11)
nbpm (5372922, 11)
rr (8636655, 11)
spo2 (8567015, 11)


In [8]:
filtered_tables['abpm'].describe()

,subject_id,hadm_id,stay_id,caregiver_id,itemid,value,valuenum,warning
count,3.096934e+06,3.096934e+06,3.096934e+06,3.096934e+06,3096934.0,3.096934e+06,3.096934e+06,3.096934e+06
mean,1.500794e+07,2.499599e+07,3.498047e+07,4.895704e+04,220052.0,7.976461e+01,7.976461e+01,5.871291e-03
std,2.897271e+06,2.868125e+06,2.884785e+06,2.939764e+04,0.0,5.637423e+02,5.637423e+02,7.639909e-02
min,1.000201e+07,2.000015e+07,3.000015e+07,1.230000e+02,220052.0,-1.350000e+02,-1.350000e+02,0.000000e+00
25%,1.248388e+07,2.248076e+07,3.249713e+07,2.287800e+04,220052.0,6.800000e+01,6.800000e+01,0.000000e+00
50%,1.502204e+07,2.503841e+07,3.495316e+07,4.632700e+04,220052.0,7.600000e+01,7.600000e+01,0.000000e+00
75%,1.750391e+07,2.743590e+07,3.748867e+07,7.605200e+04,220052.0,8.700000e+01,8.700000e+01,0.000000e+00
max,1.999944e+07,2.999962e+07,3.999955e+07,9.997000e+04,220052.0,9.300000e+05,9.300000e+05,1.000000e+00


In [20]:
filtered_tables['f']

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,10000032,29079034,39553978,18704,2180-07-23 14:00:00,2180-07-23 14:20:00,223761,98.7,98.7,°F,0
1,10000032,29079034,39553978,18704,2180-07-23 17:00:00,2180-07-23 17:04:00,223761,98.7,98.7,°F,0
2,10000032,29079034,39553978,20925,2180-07-23 20:00:00,2180-07-23 19:59:00,223761,99.5,99.5,°F,0
3,10000690,25860671,37081114,8787,2150-11-06 09:00:00,2150-11-06 09:06:00,223761,97.4,97.4,°F,0
4,10000690,25860671,37081114,8787,2150-11-06 12:00:00,2150-11-06 13:14:00,223761,97.8,97.8,°F,0
...,...,...,...,...,...,...,...,...,...,...,...
2055035,19999987,23865745,36195440,80051,2145-11-04 21:00:00,2145-11-04 21:04:00,223761,98.6,98.6,°F,0
2055036,19999987,23865745,36195440,89300,2145-11-04 08:00:00,2145-11-04 08:13:00,223761,100.6,100.6,°F,0
2055037,19999987,23865745,36195440,89300,2145-11-04 12:00:00,2145-11-04 12:17:00,223761,99.9,99.9,°F,0
2055038,19999987,23865745,36195440,89300,2145-11-04 16:00:00,2145-11-04 16:52:00,223761,98.6,98.6,°F,0


In [11]:
merged[merged['param_type']=='Numeric']['n_chartevents'].sum()

107241051

### EDA

In [4]:
print(df_icu_labdict[df_icu_labdict['label'].str.contains('glucose', case=False, na=False)])

      itemid                                label  \
64    220395                       Glucose (ingr)   
150   220621                      Glucose (serum)   
1429  225664  Glucose finger stick (range 70-100)   
1871  226537                Glucose (whole blood)   
2058  227015                     Glucose_ApacheIV   
2059  227016                GlucoseScore_ApacheIV   
2542  227976          Boost Glucose Control (1/4)   
2543  227977          Boost Glucose Control (1/2)   
2544  227978          Boost Glucose Control (3/4)   
2545  227979         Boost Glucose Control (Full)   
2883  228388         Glucose (whole blood) (soft)   
3104  228692             Glucose Control - Prophy   

                      abbreviation           linksto  \
64                  Glucose (ingr)  ingredientevents   
150                Glucose (serum)       chartevents   
1429    Glucose FS (range 70 -100)       chartevents   
1871         Glucose (whole blood)       chartevents   
2058              Glucose_Apac

In [22]:
print(df_labdict[df_labdict['label'].str.contains('glucose', case=False, na=False)])

      itemid                 label                fluid    category
7      50809               Glucose                Blood   Blood Gas
40     50842      Glucose, Ascites              Ascites   Chemistry
129    50931               Glucose                Blood   Chemistry
210    51022  Glucose, Joint Fluid          Joint Fluid   Chemistry
222    51034   Glucose, Body Fluid     Other Body Fluid   Chemistry
241    51053      Glucose, Pleural              Pleural   Chemistry
272    51084        Glucose, Urine                Urine   Chemistry
638    51478               Glucose                Urine  Hematology
906    51790          Glucose, CSF  Cerebrospinal Fluid   Chemistry
1032   51941        Glucose, Stool                Stool   Chemistry
1071   51981               Glucose                Urine   Chemistry
1117   52027  Glucose, Whole Blood                Blood   Blood Gas
1525   52569               Glucose                Blood   Chemistry


In [20]:
df_labdict.value_counts()

itemid  label                       fluid        category  
50801   Alveolar-arterial Gradient  Blood        Blood Gas     1
52005   UTX2                        Urine        Chemistry     1
52015   Xylose                      Urine        Chemistry     1
52014   Voided Specimen             Urine        Chemistry     1
52013   Vanillylmandelic Acid       Urine        Chemistry     1
                                                              ..
51383   RBC, Joint Fluid            Joint Fluid  Hematology    1
51382   Polys                       Joint Fluid  Hematology    1
51381   Other                       Joint Fluid  Hematology    1
51380   NRBC                        Joint Fluid  Hematology    1
53190   MXD%                        Blood        Chemistry     1
Length: 1646, dtype: int64

Below are the ICU itemids that are most relevant in denoting blood glucose level.

In [12]:
df_icu[(df_icu['itemid']== 220621) | (df_icu['itemid']== 225664) |
       (df_icu['itemid']== 226537) | (df_icu['itemid']== 228388)].shape

(84568, 11)

In [17]:
df_icu[(df_icu['itemid']== 220621) | (df_icu['itemid']== 225664) |
       (df_icu['itemid']== 226537) | (df_icu['itemid']== 228388)].head()

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
468,10000032,29079034,39553978,NaN,2180-07-23 21:45:00,2180-07-23 22:30:00,220621,115,115.0,mg/dL,1.0
3755,10000690,25860671,37081114,84407.0,2150-11-04 10:00:00,2150-11-04 09:39:00,225664,117,117.0,NaN,0.0
4248,10000690,25860671,37081114,NaN,2150-11-03 02:56:00,2150-11-03 04:22:00,220621,77,77.0,mg/dL,0.0
4265,10000690,25860671,37081114,NaN,2150-11-04 03:03:00,2150-11-04 03:44:00,220621,84,84.0,mg/dL,0.0
4276,10000690,25860671,37081114,NaN,2150-11-04 17:54:00,2150-11-04 18:41:00,220621,120,120.0,mg/dL,1.0


In [16]:
icu_bg = df_icu[(df_icu['itemid']== 220621) | (df_icu['itemid']== 225664) |
       (df_icu['itemid']== 226537) | (df_icu['itemid']== 228388)]
icu_bg = icu_bg.dropna(subset=['hadm_id'])
icu_bg.shape

(84568, 11)

Checks blood glucose measurements from the df_sample

In [30]:
df_subset[(df_subset['itemid']== 50931) | (df_subset['itemid']== 52569)].head(5)

,labevent_id,subject_id,hadm_id,specimen_id,itemid,order_provider_id,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
0,1,10000032,NaN,2704548,50931,P69FQC,2180-03-23 11:51:00,2180-03-23 15:56:00,___,95.0,mg/dL,70.0,100.0,NaN,ROUTINE,"IF FASTING, 70-100 NORMAL, >125 PROVISIONAL DI..."
105,106,10000032,NaN,95700408,50931,NaN,2180-05-06 22:25:00,2180-05-06 23:16:00,___,109.0,mg/dL,70.0,100.0,abnormal,STAT,"IF FASTING, 70-100 NORMAL, >125 PROVISIONAL DI..."
150,151,10000032,22595853.0,86271148,50931,NaN,2180-05-07 05:05:00,2180-05-07 07:03:00,___,99.0,mg/dL,70.0,100.0,NaN,ROUTINE,"IF FASTING, 70-100 NORMAL, >125 PROVISIONAL DI..."
176,177,10000032,NaN,19543630,50931,P85UQ1,2180-06-03 12:00:00,2180-06-03 13:04:00,___,122.0,mg/dL,70.0,100.0,abnormal,ROUTINE,"IF FASTING, 70-100 NORMAL, >125 PROVISIONAL DI..."
188,189,10000032,NaN,58691952,50931,P69FQC,2180-06-03 12:00:00,2180-06-03 13:04:00,___,127.0,mg/dL,70.0,100.0,abnormal,ROUTINE,"IF FASTING, 70-100 NORMAL, >125 PROVISIONAL DI..."
